In [42]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

#construct a list for all the long-idea csv files
filelistlong=['long'+str(i)+'.csv' for i in range(1,31)]
filelistlong

#create a dataframe to combine all the long-idea csv files
long=pd.DataFrame()
for x in filelistlong:
    long = long.append(pd.read_csv(x, header=None))

long['Strategy'] = 'Long'


In [2]:
#construct a list for all the short-idea csv files
filelistshort=['short'+str(i)+'.csv' for i in range(1,6)]
filelistshort

#create a dataframe to combine all the short-short csv files
short=pd.DataFrame()
for x in filelistshort:
    short = short.append(pd.read_csv(x, header=None))

short['Strategy'] = 'Short'

In [3]:
#combine long and short ideas to one dataframe
total = long.append(short)

#reset index and renaming columns
total = total.reset_index()
total = total.drop(['index'], axis=1)
total.columns = ['Article_Title', 'Article_Link', 'Ticker', 'Company_Name', 'Date', 'Issuer', 'Strategy']
total = total[['Ticker', 'Company_Name', 'Strategy', 'Date', 'Issuer', 'Article_Title', 'Article_Link']]


In [4]:
#write function to convert Date column values to date format
def date_converter(x):
    substr = x[:13]
    r1 = substr.replace('. ', '/')
    r2 = r1.replace(', ', '/').replace(',', '')
    r3 = r2.replace('Feb', '2').replace('Jan', '1').replace('Dec', '12').replace('Nov', '11')
    return r3

#convert Date column to new format
total.Date = total.Date.apply(date_converter)

In [5]:
#replace wrong date_time values with NaN
total.Date.loc[total.Date == total.Ticker] = np.NaN
total.Date.loc[total.Date == '•'] = np.NaN

#replace NaN with values from previous row
total = total.fillna(method = 'ffill')

#convert Date column to datetime object
def convert_datetime(x):
    try:
        return datetime.strptime(x, '%m/%d/%Y')
    except:
        pass
    
total.Date = total.Date.apply(convert_datetime)


In [13]:
total

,Ticker,Company_Name,Strategy,Date,Issuer,Article_Title,Article_Link,StartDate,EndDate,Start_Price,Week_Price,Month_Price,Quarter_Price,Semi_Price,Annual_Price
0,ISRG,"Intuitive Surgical, Inc.",Long,20170201,William Meyers,"Intuitive Surgical: China, $2 Billion Buyback ...",https://seekingalpha.com/article/4041475-intui...,20170201,20180127,wait,wait,wait,wait,wait,wait
1,MGA,Magna International Inc.,Long,20170201,Prasanna Rajagopal,Magna International: Can This Giant Grow Even ...,https://seekingalpha.com/article/4041474-magna...,20170201,20180127,wait,wait,wait,wait,wait,wait
2,LYG,Lloyds Banking Group plc,Long,20170201,Renaissance Research,Lloyds And Slowdown In U.K. Housing Market,https://seekingalpha.com/article/4041463-lloyd...,20170201,20180127,wait,wait,wait,wait,wait,wait
3,JNJ,Johnson & Johnson,Long,20170201,Tolga Kuru,The Resiliency Of J&J,https://seekingalpha.com/article/4041453-resil...,20170201,20180127,wait,wait,wait,wait,wait,wait
4,BAC,Bank of America Corporation,Long,20170201,Livermore Investing,Bank Of America Is Leveraged For Success In Ri...,https://seekingalpha.com/article/4041458-bank-...,20170201,20180127,wait,wait,wait,wait,wait,wait
5,NWQZF,BEST PACIFIC INTL HLD ORD,Long,20170201,The Value Pendulum,Best Pacific International: New Growth Drivers...,https://seekingalpha.com/article/4040983-best-...,20170201,20180127,wait,wait,wait,wait,wait,wait
6,OSTK,"Overstock.com, Inc.",Long,20170201,Jeffrey Himelson,Overstock: The New Strategy Is Working,https://seekingalpha.com/article/4041441-overs...,20170201,20180127,wait,wait,wait,wait,wait,wait
7,QTWO,"Q2 Holdings, Inc.",Long,20170201,Dan Scott,Q2 Holdings: A Growth Story Worth Paying A Pre...,https://seekingalpha.com/article/4041426-q2-ho...,20170201,20180127,wait,wait,wait,wait,wait,wait
8,MUX,McEwen Mining Inc.,Long,20170201,Tim Paul,McEwen Mining Shines Bright,https://seekingalpha.com/article/4041408-mcewe...,20170201,20180127,wait,wait,wait,wait,wait,wait
9,NVRO,Nevro,Long,20170201,Jonathan Faison,Nevro: This Medtech Runner Has More Upside,https://seekingalpha.com/article/4041389-nevro...,20170201,20180127,wait,wait,wait,wait,wait,wait


In [79]:
total.to_csv("total.csv")


In [7]:
#add dates for later calculations of stock returns over different time periods
total['StartDate']=total.Date
total['EndDate'] = total.Date + timedelta(days=360)

In [8]:
#convert Dates back to string so that yahoo_quote_download can recognize them
def date_to_string(x):
    try:
        return datetime.strftime(x, '%Y%m%d')
    except:
        pass

total.Date = total.Date.apply(date_to_string) 
total.StartDate = total.StartDate.apply(date_to_string)
total.EndDate = total.EndDate.apply(date_to_string)

In [14]:
#starting the get stock price process
from yahoo_quote_download import yqd



In [20]:
from yahoo_quote_download import yqd

for i in range(2500):
    print('currently on line:' + str(i))
    try:
        quotes = (yqd.load_yahoo_quote(total.Ticker[i], total.StartDate[i], total.EndDate[i]))
        total['Start_Price'][i] = quotes[1].split(',')[5]
        total['Week_Price'][i] = quotes[7].split(',')[5]
        total['Month_Price'][i] = quotes[23].split(',')[5]
        total['Quarter_Price'][i] = quotes[65].split(',')[5]
        total['Semi_Price'][i] = quotes[128].split(',')[5]
        total['Annual_Price'][i] = quotes[-2].split(',')[5]
        
    except:
        continue


currently on line:0
currently on line:1
currently on line:2
currently on line:3
currently on line:4
currently on line:5
currently on line:6
currently on line:7
currently on line:8
currently on line:9
currently on line:10
currently on line:11
currently on line:12
currently on line:13
currently on line:14
currently on line:15
currently on line:16
currently on line:17
currently on line:18
currently on line:19
currently on line:20
currently on line:21
currently on line:22
currently on line:23
currently on line:24
currently on line:25
currently on line:26
currently on line:27
currently on line:28
currently on line:29
currently on line:30
currently on line:31
currently on line:32
currently on line:33
currently on line:34
currently on line:35
currently on line:36
currently on line:37
currently on line:38
currently on line:39
currently on line:40
currently on line:41
currently on line:42
currently on line:43
currently on line:44
currently on line:45
currently on line:46
currently on line:47
cu

currently on line:378
currently on line:379
currently on line:380
currently on line:381
currently on line:382
currently on line:383
currently on line:384
currently on line:385
currently on line:386
currently on line:387
currently on line:388
currently on line:389
currently on line:390
currently on line:391
currently on line:392
currently on line:393
currently on line:394
currently on line:395
currently on line:396
currently on line:397
currently on line:398
currently on line:399
currently on line:400
currently on line:401
currently on line:402
currently on line:403
currently on line:404
currently on line:405
currently on line:406
currently on line:407
currently on line:408
currently on line:409
currently on line:410
currently on line:411
currently on line:412
currently on line:413
currently on line:414
currently on line:415
currently on line:416
currently on line:417
currently on line:418
currently on line:419
currently on line:420
currently on line:421
currently on line:422
currently 

currently on line:751
currently on line:752
currently on line:753
currently on line:754
currently on line:755
currently on line:756
currently on line:757
currently on line:758
currently on line:759
currently on line:760
currently on line:761
currently on line:762
currently on line:763
currently on line:764
currently on line:765
currently on line:766
currently on line:767
currently on line:768
currently on line:769
currently on line:770
currently on line:771
currently on line:772
currently on line:773
currently on line:774
currently on line:775
currently on line:776
currently on line:777
currently on line:778
currently on line:779
currently on line:780
currently on line:781
currently on line:782
currently on line:783
currently on line:784
currently on line:785
currently on line:786
currently on line:787
currently on line:788
currently on line:789
currently on line:790
currently on line:791
currently on line:792
currently on line:793
currently on line:794
currently on line:795
currently 

currently on line:1120
currently on line:1121
currently on line:1122
currently on line:1123
currently on line:1124
currently on line:1125
currently on line:1126
currently on line:1127
currently on line:1128
currently on line:1129
currently on line:1130
currently on line:1131
currently on line:1132
currently on line:1133
currently on line:1134
currently on line:1135
currently on line:1136
currently on line:1137
currently on line:1138
currently on line:1139
currently on line:1140
currently on line:1141
currently on line:1142
currently on line:1143
currently on line:1144
currently on line:1145
currently on line:1146
currently on line:1147
currently on line:1148
currently on line:1149
currently on line:1150
currently on line:1151
currently on line:1152
currently on line:1153
currently on line:1154
currently on line:1155
currently on line:1156
currently on line:1157
currently on line:1158
currently on line:1159
currently on line:1160
currently on line:1161
currently on line:1162
currently o

currently on line:1477
currently on line:1478
currently on line:1479
currently on line:1480
currently on line:1481
currently on line:1482
currently on line:1483
currently on line:1484
currently on line:1485
currently on line:1486
currently on line:1487
currently on line:1488
currently on line:1489
currently on line:1490
currently on line:1491
currently on line:1492
currently on line:1493
currently on line:1494
currently on line:1495
currently on line:1496
currently on line:1497
currently on line:1498
currently on line:1499
currently on line:1500
currently on line:1501
currently on line:1502
currently on line:1503
currently on line:1504
currently on line:1505
currently on line:1506
currently on line:1507
currently on line:1508
currently on line:1509
currently on line:1510
currently on line:1511
currently on line:1512
currently on line:1513
currently on line:1514
currently on line:1515
currently on line:1516
currently on line:1517
currently on line:1518
currently on line:1519
currently o

currently on line:1834
currently on line:1835
currently on line:1836
currently on line:1837
currently on line:1838
currently on line:1839
currently on line:1840
currently on line:1841
currently on line:1842
currently on line:1843
currently on line:1844
currently on line:1845
currently on line:1846
currently on line:1847
currently on line:1848
currently on line:1849
currently on line:1850
currently on line:1851
currently on line:1852
currently on line:1853
currently on line:1854
currently on line:1855
currently on line:1856
currently on line:1857
currently on line:1858
currently on line:1859
currently on line:1860
currently on line:1861
currently on line:1862
currently on line:1863
currently on line:1864
currently on line:1865
currently on line:1866
currently on line:1867
currently on line:1868
currently on line:1869
currently on line:1870
currently on line:1871
currently on line:1872
currently on line:1873
currently on line:1874
currently on line:1875
currently on line:1876
currently o

currently on line:2191
currently on line:2192
currently on line:2193
currently on line:2194
currently on line:2195
currently on line:2196
currently on line:2197
currently on line:2198
currently on line:2199
currently on line:2200
currently on line:2201
currently on line:2202
currently on line:2203
currently on line:2204
currently on line:2205
currently on line:2206
currently on line:2207
currently on line:2208
currently on line:2209
currently on line:2210
currently on line:2211
currently on line:2212
currently on line:2213
currently on line:2214
currently on line:2215
currently on line:2216
currently on line:2217
currently on line:2218
currently on line:2219
currently on line:2220
currently on line:2221
currently on line:2222
currently on line:2223
currently on line:2224
currently on line:2225
currently on line:2226
currently on line:2227
currently on line:2228
currently on line:2229
currently on line:2230
currently on line:2231
currently on line:2232
currently on line:2233
currently o

In [19]:
quotes = (yqd.load_yahoo_quote(total.Ticker[0], total.StartDate[0], total.EndDate[0]))
print(quotes[7].split(',')[5])
print(quotes[0])
print(quotes[-2])




235.126663
Date,Open,High,Low,Close,Adj Close,Volume
2018-01-26,437.869995,440.250000,417.000000,440.239990,440.239990,1819900


In [11]:
#add new price columns
total['Start_Price'] = 'wait'
total['Week_Price'] = 'wait'
total['Month_Price'] = 'wait'
total['Quarter_Price'] = 'wait'
total['Semi_Price'] = 'wait'
total['Annual_Price'] = 'wait'


In [22]:
total

,Ticker,Company_Name,Strategy,Date,Issuer,Article_Title,Article_Link,StartDate,EndDate,Start_Price,Week_Price,Month_Price,Quarter_Price,Semi_Price,Annual_Price
0,ISRG,"Intuitive Surgical, Inc.",Long,20170201,William Meyers,"Intuitive Surgical: China, $2 Billion Buyback ...",https://seekingalpha.com/article/4041475-intui...,20170201,20180127,232.339996,235.126663,245.233337,282.406677,311.386658,440.239990
1,MGA,Magna International Inc.,Long,20170201,Prasanna Rajagopal,Magna International: Can This Giant Grow Even ...,https://seekingalpha.com/article/4041474-magna...,20170201,20180127,42.446266,42.192333,41.870029,41.472805,46.484543,58.980000
2,LYG,Lloyds Banking Group plc,Long,20170201,Renaissance Research,Lloyds And Slowdown In U.K. Housing Market,https://seekingalpha.com/article/4041463-lloyd...,20170201,20180127,3.201614,3.211143,3.201614,3.673678,3.486547,4.130000
3,JNJ,Johnson & Johnson,Long,20170201,Tolga Kuru,The Resiliency Of J&J,https://seekingalpha.com/article/4041453-resil...,20170201,20180127,110.362480,111.190956,121.374931,121.610397,131.700592,145.330002
4,BAC,Bank of America Corporation,Long,20170201,Livermore Investing,Bank Of America Is Leveraged For Success In Ri...,https://seekingalpha.com/article/4041458-bank-...,20170201,20180127,22.533955,22.760378,24.933012,23.550591,24.143099,32.200001
5,NWQZF,BEST PACIFIC INTL HLD ORD,Long,20170201,The Value Pendulum,Best Pacific International: New Growth Drivers...,https://seekingalpha.com/article/4040983-best-...,20170201,20180127,wait,wait,wait,wait,wait,wait
6,OSTK,"Overstock.com, Inc.",Long,20170201,Jeffrey Himelson,Overstock: The New Strategy Is Working,https://seekingalpha.com/article/4041441-overs...,20170201,20180127,17.500000,17.650000,17.600000,15.400000,16.450001,74.449997
7,QTWO,"Q2 Holdings, Inc.",Long,20170201,Dan Scott,Q2 Holdings: A Growth Story Worth Paying A Pre...,https://seekingalpha.com/article/4041426-q2-ho...,20170201,20180127,31.450001,32.049999,35.299999,38.200001,37.700001,40.950001
8,MUX,McEwen Mining Inc.,Long,20170201,Tim Paul,McEwen Mining Shines Bright,https://seekingalpha.com/article/4041408-mcewe...,20170201,20180127,3.922310,4.091547,3.086081,2.658012,2.528595,2.324704
9,NVRO,Nevro,Long,20170201,Jonathan Faison,Nevro: This Medtech Runner Has More Upside,https://seekingalpha.com/article/4041389-nevro...,20170201,20180127,87.589996,93.800003,90.620003,90.599998,82.379997,81.410004


In [62]:
#clean up unavailable data
total = total.loc[total.Start_Price != 'wait']
total = total.loc[total.Start_Price != 'null']
total = total.loc[total.Week_Price != 'wait']
total = total.loc[total.Week_Price != 'null']
total = total.loc[total.Month_Price != 'wait']
total = total.loc[total.Month_Price != 'null']
total = total.loc[total.Quarter_Price != 'wait']
total = total.loc[total.Quarter_Price != 'null']
total = total.loc[total.Semi_Price != 'wait']
total = total.loc[total.Semi_Price != 'null']
total = total.loc[total.Annual_Price != 'wait']
total = total.loc[total.Annual_Price != 'null']

In [63]:
#define convert_float function
def convert_float(x):
    return float(x)

#convert all price columns into floats
total.Start_Price = total.Start_Price.apply(convert_float)
total.Week_Price = total.Week_Price.apply(convert_float)
total.Month_Price = total.Month_Price.apply(convert_float)
total.Quarter_Price = total.Quarter_Price.apply(convert_float)
total.Semi_Price = total.Semi_Price.apply(convert_float)
total.Annual_Price = total.Annual_Price.apply(convert_float)

In [49]:
import pandas as pd

In [60]:
total = pd.read_csv('total.csv', index_col = 0)

In [76]:
total = total.reset_index()

In [77]:
total = total.drop(['index'], axis=1)

In [78]:
total

,Ticker,Company_Name,Strategy,Date,Issuer,Article_Title,Article_Link,StartDate,EndDate,Start_Price,Week_Price,Month_Price,Quarter_Price,Semi_Price,Annual_Price,Week_Return,Month_Return,Quarter_Return,Semi_Return,Annual_Return
0,ISRG,"Intuitive Surgical, Inc.",Long,20170201.0,William Meyers,"Intuitive Surgical: China, $2 Billion Buyback ...",https://seekingalpha.com/article/4041475-intui...,20170201.0,20180127.0,232.339996,235.126663,245.233337,282.406677,311.386658,440.239990,0.011994,0.055493,0.215489,0.340220,0.894809
1,MGA,Magna International Inc.,Long,20170201.0,Prasanna Rajagopal,Magna International: Can This Giant Grow Even ...,https://seekingalpha.com/article/4041474-magna...,20170201.0,20180127.0,42.446266,42.192333,41.870029,41.472805,46.484543,58.980000,-0.005982,-0.013576,-0.022934,0.095139,0.389522
2,LYG,Lloyds Banking Group plc,Long,20170201.0,Renaissance Research,Lloyds And Slowdown In U.K. Housing Market,https://seekingalpha.com/article/4041463-lloyd...,20170201.0,20180127.0,3.201614,3.211143,3.201614,3.673678,3.486547,4.130000,0.002976,0.000000,0.147446,0.088997,0.289974
3,JNJ,Johnson & Johnson,Long,20170201.0,Tolga Kuru,The Resiliency Of J&J,https://seekingalpha.com/article/4041453-resil...,20170201.0,20180127.0,110.362480,111.190956,121.374931,121.610397,131.700592,145.330002,0.007507,0.099784,0.101918,0.193346,0.316842
4,BAC,Bank of America Corporation,Long,20170201.0,Livermore Investing,Bank Of America Is Leveraged For Success In Ri...,https://seekingalpha.com/article/4041458-bank-...,20170201.0,20180127.0,22.533955,22.760378,24.933012,23.550591,24.143099,32.200001,0.010048,0.106464,0.045116,0.071410,0.428955
5,OSTK,"Overstock.com, Inc.",Long,20170201.0,Jeffrey Himelson,Overstock: The New Strategy Is Working,https://seekingalpha.com/article/4041441-overs...,20170201.0,20180127.0,17.500000,17.650000,17.600000,15.400000,16.450001,74.449997,0.008571,0.005714,-0.120000,-0.060000,3.254286
6,QTWO,"Q2 Holdings, Inc.",Long,20170201.0,Dan Scott,Q2 Holdings: A Growth Story Worth Paying A Pre...,https://seekingalpha.com/article/4041426-q2-ho...,20170201.0,20180127.0,31.450001,32.049999,35.299999,38.200001,37.700001,40.950001,0.019078,0.122416,0.214626,0.198728,0.302067
7,MUX,McEwen Mining Inc.,Long,20170201.0,Tim Paul,McEwen Mining Shines Bright,https://seekingalpha.com/article/4041408-mcewe...,20170201.0,20180127.0,3.922310,4.091547,3.086081,2.658012,2.528595,2.324704,0.043147,-0.213198,-0.322335,-0.355330,-0.407313
8,NVRO,Nevro,Long,20170201.0,Jonathan Faison,Nevro: This Medtech Runner Has More Upside,https://seekingalpha.com/article/4041389-nevro...,20170201.0,20180127.0,87.589996,93.800003,90.620003,90.599998,82.379997,81.410004,0.070899,0.034593,0.034365,-0.059482,-0.070556
9,DERM,Dermira,Long,20170201.0,Jonathan Faison,Dermira's Late-Stage Assets Are Undervalued,https://seekingalpha.com/article/4041386-dermi...,20170201.0,20180127.0,29.330000,29.010000,38.029999,32.220001,26.719999,30.930000,-0.010910,0.296625,0.098534,-0.088987,0.054552


In [1]:
total['Week_Return'] = total.Week_Price/total.Start_Price-1
total['Month_Return'] = total.Month_Price/total.Start_Price-1
total['Quarter_Return'] = total.Quarter_Price/total.Start_Price-1
total['Semi_Return'] = total.Semi_Price/total.Start_Price-1
total['Annual_Return'] = total.Annual_Price/total.Start_Price-1


NameError: name 'total' is not defined

In [71]:
total

,Ticker,Company_Name,Strategy,Date,Issuer,Article_Title,Article_Link,StartDate,EndDate,Start_Price,Week_Price,Month_Price,Quarter_Price,Semi_Price,Annual_Price,Week_Return,Month_Return,Quarter_Return,Semi_Return,Annual_Return
0,ISRG,"Intuitive Surgical, Inc.",Long,20170201.0,William Meyers,"Intuitive Surgical: China, $2 Billion Buyback ...",https://seekingalpha.com/article/4041475-intui...,20170201.0,20180127.0,232.339996,235.126663,245.233337,282.406677,311.386658,440.239990,0.011994,0.055493,0.215489,0.340220,0.894809
1,MGA,Magna International Inc.,Long,20170201.0,Prasanna Rajagopal,Magna International: Can This Giant Grow Even ...,https://seekingalpha.com/article/4041474-magna...,20170201.0,20180127.0,42.446266,42.192333,41.870029,41.472805,46.484543,58.980000,-0.005982,-0.013576,-0.022934,0.095139,0.389522
2,LYG,Lloyds Banking Group plc,Long,20170201.0,Renaissance Research,Lloyds And Slowdown In U.K. Housing Market,https://seekingalpha.com/article/4041463-lloyd...,20170201.0,20180127.0,3.201614,3.211143,3.201614,3.673678,3.486547,4.130000,0.002976,0.000000,0.147446,0.088997,0.289974
3,JNJ,Johnson & Johnson,Long,20170201.0,Tolga Kuru,The Resiliency Of J&J,https://seekingalpha.com/article/4041453-resil...,20170201.0,20180127.0,110.362480,111.190956,121.374931,121.610397,131.700592,145.330002,0.007507,0.099784,0.101918,0.193346,0.316842
4,BAC,Bank of America Corporation,Long,20170201.0,Livermore Investing,Bank Of America Is Leveraged For Success In Ri...,https://seekingalpha.com/article/4041458-bank-...,20170201.0,20180127.0,22.533955,22.760378,24.933012,23.550591,24.143099,32.200001,0.010048,0.106464,0.045116,0.071410,0.428955
5,OSTK,"Overstock.com, Inc.",Long,20170201.0,Jeffrey Himelson,Overstock: The New Strategy Is Working,https://seekingalpha.com/article/4041441-overs...,20170201.0,20180127.0,17.500000,17.650000,17.600000,15.400000,16.450001,74.449997,0.008571,0.005714,-0.120000,-0.060000,3.254286
6,QTWO,"Q2 Holdings, Inc.",Long,20170201.0,Dan Scott,Q2 Holdings: A Growth Story Worth Paying A Pre...,https://seekingalpha.com/article/4041426-q2-ho...,20170201.0,20180127.0,31.450001,32.049999,35.299999,38.200001,37.700001,40.950001,0.019078,0.122416,0.214626,0.198728,0.302067
7,MUX,McEwen Mining Inc.,Long,20170201.0,Tim Paul,McEwen Mining Shines Bright,https://seekingalpha.com/article/4041408-mcewe...,20170201.0,20180127.0,3.922310,4.091547,3.086081,2.658012,2.528595,2.324704,0.043147,-0.213198,-0.322335,-0.355330,-0.407313
8,NVRO,Nevro,Long,20170201.0,Jonathan Faison,Nevro: This Medtech Runner Has More Upside,https://seekingalpha.com/article/4041389-nevro...,20170201.0,20180127.0,87.589996,93.800003,90.620003,90.599998,82.379997,81.410004,0.070899,0.034593,0.034365,-0.059482,-0.070556
9,DERM,Dermira,Long,20170201.0,Jonathan Faison,Dermira's Late-Stage Assets Are Undervalued,https://seekingalpha.com/article/4041386-dermi...,20170201.0,20180127.0,29.330000,29.010000,38.029999,32.220001,26.719999,30.930000,-0.010910,0.296625,0.098534,-0.088987,0.054552


In [73]:
total = total.loc[total.Start_Price > 0]

In [80]:
total

,Ticker,Company_Name,Strategy,Date,Issuer,Article_Title,Article_Link,StartDate,EndDate,Start_Price,Week_Price,Month_Price,Quarter_Price,Semi_Price,Annual_Price,Week_Return,Month_Return,Quarter_Return,Semi_Return,Annual_Return
0,ISRG,"Intuitive Surgical, Inc.",Long,20170201.0,William Meyers,"Intuitive Surgical: China, $2 Billion Buyback ...",https://seekingalpha.com/article/4041475-intui...,20170201.0,20180127.0,232.339996,235.126663,245.233337,282.406677,311.386658,440.239990,0.011994,0.055493,0.215489,0.340220,0.894809
1,MGA,Magna International Inc.,Long,20170201.0,Prasanna Rajagopal,Magna International: Can This Giant Grow Even ...,https://seekingalpha.com/article/4041474-magna...,20170201.0,20180127.0,42.446266,42.192333,41.870029,41.472805,46.484543,58.980000,-0.005982,-0.013576,-0.022934,0.095139,0.389522
2,LYG,Lloyds Banking Group plc,Long,20170201.0,Renaissance Research,Lloyds And Slowdown In U.K. Housing Market,https://seekingalpha.com/article/4041463-lloyd...,20170201.0,20180127.0,3.201614,3.211143,3.201614,3.673678,3.486547,4.130000,0.002976,0.000000,0.147446,0.088997,0.289974
3,JNJ,Johnson & Johnson,Long,20170201.0,Tolga Kuru,The Resiliency Of J&J,https://seekingalpha.com/article/4041453-resil...,20170201.0,20180127.0,110.362480,111.190956,121.374931,121.610397,131.700592,145.330002,0.007507,0.099784,0.101918,0.193346,0.316842
4,BAC,Bank of America Corporation,Long,20170201.0,Livermore Investing,Bank Of America Is Leveraged For Success In Ri...,https://seekingalpha.com/article/4041458-bank-...,20170201.0,20180127.0,22.533955,22.760378,24.933012,23.550591,24.143099,32.200001,0.010048,0.106464,0.045116,0.071410,0.428955
5,OSTK,"Overstock.com, Inc.",Long,20170201.0,Jeffrey Himelson,Overstock: The New Strategy Is Working,https://seekingalpha.com/article/4041441-overs...,20170201.0,20180127.0,17.500000,17.650000,17.600000,15.400000,16.450001,74.449997,0.008571,0.005714,-0.120000,-0.060000,3.254286
6,QTWO,"Q2 Holdings, Inc.",Long,20170201.0,Dan Scott,Q2 Holdings: A Growth Story Worth Paying A Pre...,https://seekingalpha.com/article/4041426-q2-ho...,20170201.0,20180127.0,31.450001,32.049999,35.299999,38.200001,37.700001,40.950001,0.019078,0.122416,0.214626,0.198728,0.302067
7,MUX,McEwen Mining Inc.,Long,20170201.0,Tim Paul,McEwen Mining Shines Bright,https://seekingalpha.com/article/4041408-mcewe...,20170201.0,20180127.0,3.922310,4.091547,3.086081,2.658012,2.528595,2.324704,0.043147,-0.213198,-0.322335,-0.355330,-0.407313
8,NVRO,Nevro,Long,20170201.0,Jonathan Faison,Nevro: This Medtech Runner Has More Upside,https://seekingalpha.com/article/4041389-nevro...,20170201.0,20180127.0,87.589996,93.800003,90.620003,90.599998,82.379997,81.410004,0.070899,0.034593,0.034365,-0.059482,-0.070556
9,DERM,Dermira,Long,20170201.0,Jonathan Faison,Dermira's Late-Stage Assets Are Undervalued,https://seekingalpha.com/article/4041386-dermi...,20170201.0,20180127.0,29.330000,29.010000,38.029999,32.220001,26.719999,30.930000,-0.010910,0.296625,0.098534,-0.088987,0.054552
